# Homework for week04

> In this homework, we will use the Car price dataset like last week.
>
> We'll work with the MSRP variable, and we'll transform it to a classification task.

For the rest of the homework, you'll need to use only these columns:

- Make,
- Model,
- Year,
- Engine HP,
- Engine Cylinders,
- Transmission Type,
- Vehicle Style,
- highway MPG,
- city mpg
- MSRP

## Import packages

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score

## Data Preparation

In [ ]:
def load_prepare_data():

    # load the data
    print(f'Reading csv file into dataframe....')
    df = pd.read_csv('../data/data-hmwk-3.csv')
    print(f'Original df.shape: {df.shape}')

    # clean/prepare the data
    print(f'cleaning data....')
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    df.columns = df.columns.str.replace('msrp', 'price')
    
    # remove duplicates
    # print(f'removing duplicated records....')
    # df.drop_duplicates(inplace=True)

    # identify features and target
    features = ['make', 'model', 'year', 'engine_hp', 'engine_cylinders',  
                      'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg'
                    ]
    target = ['price']

    # use a subset for analysis
    df = df[features+target]

    # impute Nan/nulls with 0
    print(f'imputing NULLs....')
    df = df.fillna(0)

    # calculate the mean for price column
    print(f'binarizing target series....')
    mean_price = df.price.mean()
    # create a new column called 'above_average' that is set to 1 if value is above the mean_price
    df['above_average'] = [1 if price > mean_price else 0 for price in df['price']] 

    # remove price as target and replace with 'above_average'
    del df['price']
    target = ['above_average']

    return df


In [ ]:
data = load_prepare_data()

In [ ]:
data.head().T

In [ ]:
data.describe()

### Split the data

In [ ]:
# split the data to train/val/test sets with 60%/20%/20% distribution
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

# set the y dataframe
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

# remove target from df
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

## Question 1: ROC AUC feature importance

> ROC AUC could also be used to evaluate feature importance of numerical variables.
> 
>Let's do that
> 
>     For each numerical variable, use it as score and compute AUC with the above_average variable
>     Use the training dataset for that
> 
> If your AUC is < 0.5, invert this variable by putting "-" in front
> 
> (e.g. -df_train['engine_hp'])
> 
> AUC can go below 0.5 if the variable is negatively correlated with the target varialble. You can change > the direction of the correlation by negating this variable - then negative correlation becomes positive.

In [ ]:
df_full_train.info()

In [ ]:
categorical_columns = list(df_full_train.select_dtypes(include=['category', 'object']).columns)
numerical_columns = list(df_full_train.select_dtypes(include=['number']).columns)

In [ ]:
categorical_columns

In [ ]:
numerical_columns

In [ ]:
df_full_train[['engine_hp', 'highway_mpg', 'city_mpg', 'above_average']].reset_index(drop=True).head()

In [ ]:
df_full_train[numerical_columns]

In [ ]:
df_full_train.above_average.value_counts()

In [ ]:
df_full_train.above_average.mean()

In [ ]:
sns.heatmap(df_full_train[numerical_columns].corr(), annot=True, fmt='.2f', cmap='RdBu_r', square=True)
plt.title('Bivariate analysis: heatmap of correlation of variables')
plt.show()

In [ ]:
for col in ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']:
    auc_score = roc_auc_score(df_full_train['above_average'], df_full_train[col])
    if auc_score < 0.5 :
        auc_score = roc_auc_score(df_full_train['above_average'], -df_full_train[col])
    print(f'AUC score for {col}: {auc_score:.2f}')


### notes
(auc_score < 0.5) condition not checked, we get below variables < 0.5

> AUC score for highway_mpg: 0.37<br>
> AUC score for city_mpg: 0.33

Answers with duplicates dropped, which is not part of instructions, is similarly close. Qn2's answer however is different when duplicates are dropped, so the code is removed/commented for now.

```python
df.drop_duplicates(inplace=True)
```
> AUC score for engine_hp: 0.92<br>
> AUC score for engine_cylinders: 0.77<br>
> AUC score for highway_mpg: 0.64<br>
> AUC score for city_mpg: 0.68<br>


### qn1 ans: `engine_hp`

q: Which numerical variable (among the following 4) has the highest AUC?

a: `engine_hp`


## Question 2: Training the model

> Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:

> LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)

In [ ]:
def perform_ohe(df_train, df_val):
    # perform OHE on categorical data; made sparse due to non-convergence otherwise
    dv = DictVectorizer(sparse=True)

    train = df_train.reset_index(drop=True)
    val = df_val.reset_index(drop=True)

    train_dict = train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = val.to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # print(X.shape, y.shape)
    return X_train, X_val

In [ ]:
X_train, X_val = perform_ohe(df_train, df_val)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred)
print(f'{auc.round(4) }')            # `0.970` when duplicates dropped
print(f'{auc.round(4) - 0.979}') # this is the closest answer
print(f'{auc.round(4) - 0.878}')

### qn2 ans: `0.979`

Q: What's the AUC of this model on the validation dataset? (round to 3 digits)

A: `0.979`

## Question 3: Precision and Recall

> Now let's compute precision and recall for our model.
>
>    - Evaluate the model on all thresholds from 0.0 to 1.0 with step 0.01
>    - For each threshold, compute precision and recall
>    - Plot them


In [ ]:
# code is from lesson
scores = []

thresholds = np.linspace(0, 1, 101)

for t in thresholds:
    actual_positive = (y_val == 1)
    actual_negative = (y_val == 0)
    
    predict_positive = (y_pred >= t)
    predict_negative = (y_pred < t)

    tp = (predict_positive & actual_positive).sum()
    tn = (predict_negative & actual_negative).sum()

    fp = (predict_positive & actual_negative).sum()
    fn = (predict_negative & actual_positive).sum()
    
    scores.append((t, tp, fp, fn, tn))

In [ ]:
# code is from lesson
columns = ['threshold', 'tp', 'fp', 'fn', 'tn']
df_scores = pd.DataFrame(scores, columns=columns)

df_scores['tpr'] = df_scores.tp / (df_scores.tp + df_scores.fn)
df_scores['fpr'] = df_scores.fp / (df_scores.fp + df_scores.tn)

In [ ]:
# code is from lesson
# plt.plot(df_scores.threshold, df_scores['tpr'], label='TPR')
# plt.plot(df_scores.threshold, df_scores['fpr'], label='FPR')
# plt.legend()

Definitions:

- _precision_ : fraction of **POSITIVE** predictions that are **CORRECT**
$$precision = tp / (tp + fp)$$
- _recall_ : fraction of **CORRECT**ly identified **POSITIVE** samples
$$recall = tp / (tp + fn)$$
- _fpr_ : False Positive Rate
$$fpr = fp / (tn + fp)$$
- _tpr_ : True Positive Rate
$$tpr = fn / (fn + tp)$$

In [ ]:
# code is from lesson
df_scores['p'] = df_scores.tp / (df_scores.tp + df_scores.fp)
df_scores['r'] = df_scores.tp / (df_scores.tp + df_scores.fn)

In [ ]:
# code is from lesson
plt.plot(df_scores.threshold, df_scores.p, label='precision')
plt.plot(df_scores.threshold, df_scores.r, label='recall')

plt.vlines(0.48, 0, 1, color='grey', linestyle='--', alpha=0.5)
plt.annotate("0.48",  (0.5,0.87))
plt.legend()
plt.show()

### qn3 ans: `0.48`

Q: At which threshold precision and recall curves intersect?

A: `0.48` (drop_duplicates gives 0.44)


## Question 4: F1 score

> Precision and recall are conflicting - when one grows, the other goes down. That's why they are often combined into the F1 score - a metrics that takes into account both

> This is the formula for computing F1:

$$F_1 = 2 \cdot \cfrac{P \cdot R}{P + R}$$

> Where $P$ is precision and $R$ is recall.

> Let's compute F1 for all thresholds from 0.0 to 1.0 with increment 0.01

In [ ]:
# use above formula and add to same dataframe
df_scores['f1'] = 2 * (df_scores.p * df_scores.r / (df_scores.p + df_scores.r))

In [ ]:
df_scores[::10]

In [ ]:
print(f'{df_scores["f1"].max() = }')

# idx is in tens and thresholds in 1/100, so multiply idx*001 to get threshold value
max_f1_threshold = df_scores['f1'].idxmax()*0.01 
print(f'{max_f1_threshold = }')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

line = ax.plot(df_scores.threshold, df_scores.f1)
dash = ax.vlines((0.12, 0.32, 0.52, 0.72), 0.0, 0.95, color='grey', linestyle='--', alpha=0.5)
ax.annotate('0.12',
            xy=(0.12, 0.8), xycoords='data',
            xytext=(0.12, 0.68), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='right', verticalalignment='top')
ax.annotate('0.32',
            xy=(0.32, 0.88), xycoords='data',
            xytext=(0.32, 0.78), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='right', verticalalignment='top')
ax.annotate('0.49 (mine)',
            xy=(0.49, 0.88), xycoords='data',
            xytext=(0.49, 0.75), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='right', verticalalignment='top')
ax.annotate('0.52 (ans)',
            xy=(0.52, 0.88), xycoords='data',
            xytext=(0.54, 0.75), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='left', verticalalignment='bottom')
ax.annotate('0.72',
            xy=(0.72, 0.85), xycoords='data',
            xytext=(0.72, 0.75), textcoords='axes fraction',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='left', verticalalignment='top')
plt.xticks(np.linspace(0, 0.9, 11))
plt.yticks(np.linspace(0.0, 1.0, 5))
plt.show()

### qn4 ans: `0.52`

Q: At which threshold F1 is maximal?

A: `0.49` take closest `0.52`

## Question 5: 5-Fold CV

Use the `KFold` class from Scikit-Learn to evaluate our model on 5 different folds:

```
KFold(n_splits=5, shuffle=True, random_state=1)
```

* Iterate over different folds of `df_full_train`
* Split the data into train and validation
* Train the model on train with these parameters: `LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)`
* Use AUC to evaluate the model on validation

In [ ]:
categorical_columns

In [ ]:
numerical_columns

In [ ]:
numerical_columns.remove('above_average')

In [ ]:
def train(df_train, y_train):
    dicts = df_train[categorical_columns + numerical_columns].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

def predict_prob(df, dv, model):
    dicts = df[categorical_columns + numerical_columns].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

def predict(df, dv, model):
    dicts = df[categorical_columns + numerical_columns].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict(X)

    return y_pred

In [ ]:
scores = []
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

In [ ]:
for train_idx, val_idx in tqdm(kfold.split(df_full_train)):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    # set the y dataframe
    y_train = df_train.above_average.values
    y_val = df_val.above_average.values

    # remove target from df
    del df_train['above_average']
    del df_val['above_average']

    dv, model = train(df_train, y_train)
    y_pred = predict_prob(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print(f'mean auc: {np.mean(scores):.3f}, std: +-{np.std(scores):.3f}')

### qn5 ans: `0.003`

Q: How large is standard devidation of the scores across different folds?

A: `0.004`

## Question 6: Hyperparameter Tuning

Now let's use 5-Fold cross-validation to find the best parameter C

- Iterate over the following C values: [0.01, 0.1, 0.5, 10]
- Initialize KFold with the same parameters as previously
- Use these parametes for the model: LogisticRegression(solver='liblinear', C=C, max_iter=1000)
- Compute the mean score as well as the std (round the mean and std to 3 decimal digits)


In [ ]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical_columns + numerical_columns].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [ ]:
scores_dict = {}
for C in tqdm([0.01, 0.1, 0.5, 10]):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    scores = []

    for train_idx, val_idx in kfold.split(df_full_train):
        df_train = df_full_train.iloc[train_idx]
        df_val = df_full_train.iloc[val_idx]

        # set the y dataframe
        y_train = df_train.above_average.values
        y_val = df_val.above_average.values

        # remove target from df
        del df_train['above_average']
        del df_val['above_average']

        dv, model = train(df_train, y_train, C=C)
        y_pred = predict_prob(df_val, dv, model)

        auc = roc_auc_score(y_val, y_pred)
        f1 = f1_score(y_val, predict(df_val, dv, model))
        scores.append(auc)
        
    print(f'loop, C= {C}, mean auc: {np.mean(scores)}, f1 score: {f1}')
        
    scores_dict[repr(C)] = { 'mean auc' : np.mean(scores).round(3)
                                          , 'std var' : '+-'+repr(np.std(scores).round(5))
                                    }


In [ ]:
from pprint import pprint

pprint(scores_dict, indent=5)


### qn6 ans: `10`

Q: Which C leads to the best mean score?

A: `10`

## Extras

### EDA

- what if we change 'year', 'engine_cylinders',  'above_average' variables to categorical variables, would the answers differ?

In [ ]:
data = load_prepare_data()
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
df_full_train[['year', 'engine_cylinders', 'above_average']] = df_full_train[['year'
                                                                              , 'engine_cylinders'
                                                                              ,  'above_average']].astype('category')

In [ ]:
df_full_train.info()

In [ ]:
df_full_train.describe(percentiles=[0.1, .25, .5, .75, 0.99])

It's evident from the `min` to `10th` percentile and `99th` percentile to `max`, our dataset has outliers.

In [ ]:
categorical_columns = list(df_full_train.select_dtypes(include=['category', 'object']).columns)
numerical_columns = list(df_full_train.select_dtypes(include=['number']).columns)

In [ ]:
sns.pairplot(df_full_train, hue='above_average',  diag_kind="hist", corner=True)
plt.title('pairplot of numerical variables')
plt.show()

`city_mpg` and `highway_mpg` has long tail, and outliers as already discovered above, and confirmed by the `boxplot` below.

Not just needing to treat outliers and long-tail distribution, we also have to scale the numerical data as the y-axes show that they have different values.

In [ ]:
df_full_train.plot.box(figsize=(12, 12), layout=(3,3), subplots=True, sharex=False, sharey=False)
plt.show()

In [ ]:
sns.stripplot(df_full_train, x="city_mpg", y="engine_hp", hue="above_average", orient='h', dodge=True)
plt.title('city_mpg vs engine_hp by above_average breakdown')
plt.yticks([])
plt.show()

In [ ]:
categorical_columns.remove('above_average')

In [ ]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical_columns + numerical_columns].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

def predict_prob(df, dv, model):
    dicts = df[categorical_columns + numerical_columns].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

def predict(df, dv, model):
    dicts = df[categorical_columns + numerical_columns].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict(X)

    return y_pred

In [ ]:
scores = []
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

In [ ]:
scores_dict = {}
for C in tqdm([0.01, 0.1, 0.5, 10]):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    scores = []

    for train_idx, val_idx in kfold.split(df_full_train):
        df_train = df_full_train.iloc[train_idx]
        df_val = df_full_train.iloc[val_idx]
        # display(df_train.head())

        # set the y dataframe
        y_train = df_train.above_average.values
        y_val = df_val.above_average.values

        # remove target from df
        del df_train['above_average']
        del df_val['above_average']
        # display(df_train.head())

        dv, model = train(df_train, y_train, C=C)
        y_pred = predict_prob(df_val, dv, model)

        auc = roc_auc_score(y_val, y_pred)
        f1 = f1_score(y_val, predict(df_val, dv, model))
        scores.append(auc)
        scores.append(f1)

    print(f'loop, C= {C}, mean auc: {np.mean(scores)}, f1 score: {f1}')
        
    scores_dict[repr(C)] = { 'mean auc' : np.mean(scores).round(3)
                                          , 'std var' : '+-'+repr(np.std(scores).round(5))
                                          , 'mean f1' : np.mean(scores).round(5)
                                    }


In [ ]:
X = df_train[['city_mpg', 'engine_hp']]
dicts = X.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts)

model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
model.fit(X_train, y_train)


In [ ]:
X.info()

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
_, ax = plt.subplots(figsize=(4, 3))
DecisionBoundaryDisplay.from_estimator(
    model,
    X,
    cmap=plt.cm.Paired,
    ax=ax,
    response_method="predict",
    plot_method="pcolormesh",
    shading="auto",
    xlabel="city_mpg",
    ylabel="engine_hp",
    eps=0.5,
)

# Plot also the training points
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y_train, edgecolors="k", cmap=plt.cm.Paired)

plt.xticks(())
plt.yticks(())

plt.show()